In [13]:
from autogen import ConversableAgent

In [17]:
phi3 = {
    "config_list": [
        {
            "model": "microsoft/Phi-3-mini-4k-instruct-gguf",
            "base_url": "http://172.31.80.1:5506/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

llama3_8b = {
    "config_list": [
        {
            "model": "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
            "base_url": "http://172.31.80.1:5506/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

llama3_42b = {
    "config_list": [
        {
            "model": "InferenceIllusionist/llama3-42b-v0-iMat-GGUF",
            "base_url": "http://172.31.80.1:5506/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

In [21]:
llm=llama3_8b

In [39]:
jack = ConversableAgent(
    "Jack",
    llm_config=llm,
    system_message="Your name is Jack and you are a comedian in a two-person comedy show.",
    human_input_mode='NEVER',
    max_consecutive_auto_reply=1
    #is_termination_msg=lambda msg: "good bye" or "goodbye" in msg["content"].lower(),
)
adriana = ConversableAgent(
    "Adriana",
    llm_config=llm,
    system_message="Your name is Adriana and you are a comedian in two-person comedy show.",
    human_input_mode='NEVER',
    max_consecutive_auto_reply=3
    
)

In [40]:
chat_result = jack.initiate_chat(adriana, message="Adriana, tell me a joke.")

Jack (to Adriana):

Adriana, tell me a joke.

--------------------------------------------------------------------------------
Adriana (to Jack):

Here's one: Why don't scientists trust atoms? Because they make up everything! (get it?) *wink* My partner in crime, Alex, is probably rolling his eyes at my terrible puns right now... but hey, someone's gotta keep the laughter alive! What do you think?

--------------------------------------------------------------------------------
Jack (to Adriana):

*laughs* Ahah, that's a great one, Adriana! I love it! *plays along* Yeah, Alex might be groaning in the background, but we know our audience is loving it! You're killing it out there!

Now, my turn! Why did the scarecrow win an award? Because he was outstanding in his field! *wink*

What do you think, folks? Do we have a pun-filled duo on our hands?

--------------------------------------------------------------------------------
Adriana (to Jack):

*laughs* Oh, burn! I mean, nice one! *appl

In [48]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the number 25 in your mind, and I will try to guess it. If I guess too high, say 'too high', if I guess too low, say 'too low'.",
    llm_config=llm,
    is_termination_msg=lambda msg: "25" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE", 
    max_consecutive_auto_reply=1
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config=llm,
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 50. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 50. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Sounds like fun!

My first guess is... 25!

Is my guess too high or too low?

--------------------------------------------------------------------------------


Please give feedback to agent_guess_number. Press enter or type 'exit' to stop the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


Code Executor

In [69]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [58]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

# Generate a reply for the given code.
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)

Provide feedback to the sender. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to scatter.png



In [83]:
import os

print(os.listdir(temp_dir.name))

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmppq2segkt'

In [82]:
temp_dir.cleanup()

Code Executor - Docker

In [86]:
from autogen.coding import DockerCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

# When the code executor is no longer used, stop it to release the resources.
# executor.stop()

In [87]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

# Generate a reply for the given code.
reply = code_executor_agent_using_docker.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)

Provide feedback to the sender. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 1 (execution failed)
Code output: Traceback (most recent call last):
  File "/workspace/tmp_code_e24bf32d4a21990fb9e4b5eb889ebe5a.py", line 1, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'



Code Writer and Executor Agent

In [94]:
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. 
After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config=llm,
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [95]:
# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [98]:
chat_result = code_executor_agent_using_docker.initiate_chat(
    code_writer_agent,
    message="Write Python code to calculate the 14th Fibonacci number.",
)

code_executor_agent_docker (to code_writer_agent):

Write Python code to calculate the 14th Fibonacci number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent_docker):

Here's a simple Python function that calculates the nth Fibonacci number using recursion:

```
sh
def fibonacci(n):
    if n <= 0:
        return "Input should be positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

However, this recursive solution is not efficient for large values of `n` because it does a lot of repeated computation. We can use an iterative approach to solve this problem efficiently:

```
sh
def fibonacci(n):
    if n <= 0:
        return "Input should be positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
     

Provide feedback to code_writer_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING 3 CODE BLOCKS (inferred languages are [python, python, python])...
code_executor_agent_docker (to code_writer_agent):

exitcode: 1 (execution failed)
Code output: Traceback (most recent call last):
  File "/workspace/tmp_code_c01864e760833511c3079490d8ca8c8b.py", line 1, in <module>
    sh
NameError: name 'sh' is not defined


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent_docker):

I apologize for the mistake. The code I provided earlier has a syntax error because it contains an unnecessary `sh` block at the beginning.

Here's the corrected Python code to calculate the 14th Fibonacci number using recursion:
```
def fibonacci(n):
    if n <= 0:
        return "Input should be positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-

Provide feedback to code_writer_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING 2 CODE BLOCKS (inferred languages are [python, python])...
code_executor_agent_docker (to code_writer_agent):

exitcode: 0 (execution succeeded)
Code output: 233
233


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent_docker):

The code I provided earlier correctly calculates the 14th Fibonacci number as 233.

Here's a breakdown of how the calculation works:

* The Fibonacci sequence starts with 0 and 1.
* To calculate the next number in the sequence, you add the previous two numbers together. For example, to get the third number (2), you add the first number (0) and the second number (1).
* This process is repeated until you reach the desired number.

In this case, we are calculating the 14th Fibonacci number, which is `fibonacci(14)`.

The recursive function calculates the 14th number by adding the 

Provide feedback to code_writer_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [99]:
temp_dir.cleanup()

In [100]:
executor.stop()